** Sample rows from source file (page-views.csv) **

1496253932,/search?q=geophysics,,1689<br>
1496253946,/repository/1207,/search?q=geophysics,1689<br>
1496253979,/repository/8057,/search?q=geophysics,1689<br>
1496229649,/search?q=millstream,,7040<br>
1496229668,/repository/8947,/search?q=millstream,7040<br>
1496229692,/repository/7436,/search?q=millstream,7040<br>
1496229666,/repository/4531,/search?q=millstream,7040<br>
1496280938,/search?q=sibyllist,,4190<br>
1496280958,/repository/8561,/search?q=sibyllist,4190<br>
1496280949,/repository/1748,/search?q=sibyllist,4190<br>
1496256457,/search?q=protocanonical,,214<br>
1496256488,/repository/4345,/search?q=protocanonical,214<br>
1496256490,/repository/2128,/search?q=protocanonical,214<br>
1496256514,/repository/9783,/search?q=protocanonical,214<br>
1496259900,/search?q=kerchunk,,2340<br>
1496259937,/repository/6248,/search?q=kerchunk,2340<br>
1496259941,/repository/5532,/search?q=kerchunk,2340<br>
1496278788,/search?q=postulate,,1614<br>
1496278829,/repository/1357,/search?q=postulate,1614<br>
1496278811,/repository/3555,/search?q=postulate,1614<br>

In [1]:
# Import Package(s)
import pandas as pd

# Set csv_path: The path of the source file.
csv_path = 'C:\Temp\page-views.csv'

# Set top_N: The number of top records to return.
top_N = 5

pd.options.mode.chained_assignment = None

In [2]:
# Create function to load page views CSV into dataframe.
def get_page_views(csv_path):

    # Load page views CSV into df_page_views dataframe.
    df_page_views = pd.read_csv(csv_path, header=None, names=['timestamp', 'path', 'referrer', 'cid'])
    
    # Replace NaN referrer values with empty string.
    df_page_views['referrer'] = df_page_views['referrer'].fillna('')
    
    # Add query field to df_page_views data frame.
    df_page_views = set_page_views_query(df_page_views)

    return(df_page_views)

In [3]:
# Create function to create query field in data frame, using both path and referrer fields.
def set_page_views_query(df_page_views):

    # Create df_page_views_path data frame containing only relevant rows, filtering on path.
    df_page_views_path = df_page_views[df_page_views['path'].str.startswith('/search?q=')]

    # Create query column based on content of path.
    df_page_views_path['query'] = df_page_views_path['path'].apply(lambda x: x.replace('/search?q=', '').strip())
    
    # Create df_page_views_referrer data frame containing only relevant rows, filtering on referrer.
    df_page_views_referrer = df_page_views[df_page_views['referrer'].str.startswith('/search?q=')]

    # Create query column based on content of referrer.
    df_page_views_referrer['query'] = df_page_views_referrer['referrer'].apply(lambda x: x.replace('/search?q=', '').strip())

    # Concatenate dataframes then return combined dataframe.
    return(pd.concat([df_page_views_path, df_page_views_referrer]))

In [4]:
# Create function to compute query counts for both issued queries and query referrals.
def get_top_query_counts(df_page_views, group_by, top_N = 5):
    
    #group_by = issued:
    # Identify most frequently issued queries, including query counts.
    
    #group_by = referral:
    # Indentify top queries by number of referrals, including click counts.
    
    # top:
    # The number of top queries to return. Defaults to 5.
    
    # Validate group_by argument.
    if (group_by not in ['issued', 'referral']):
        print("Invalid 'group_by' value supplied. Please use 'issued' or referral'")
        return()
        
    # Set primary field.
    query_source_field = ''
    if (group_by == 'issued'):
        query_source_field = 'path'
    else:
        query_source_field = 'referrer'

    # Create df_filtered_page_views data frame containing only relevant rows:
    # If group_by = 'issued' then filter on path field.
    # If group_by = 'referral' then filter on referrer field.
    df_filtered_page_views = df_page_views[df_page_views[query_source_field].str.startswith('/search?q=')]
    
    # Group by query.
    df_filtered_page_views_groupby_query = df_filtered_page_views.groupby('query')[['query']].count()

    # Rename query column to count.
    df_filtered_page_views_groupby_query.columns = ['count']

    # Sort by count in descending order.
    df_filtered_page_views_groupby_query = df_filtered_page_views_groupby_query.sort_values(by = 'count', ascending=False)

    # Reset the index.
    df_filtered_page_views_groupby_query = df_filtered_page_views_groupby_query.reset_index()
    
    # Create rank column.
    df_filtered_page_views_groupby_query['count_rank'] = df_filtered_page_views_groupby_query['count'].rank()
    
    # Retrieve the top 5 query count ranks.
    lst_top_query_count_ranks = sorted(df_filtered_page_views_groupby_query['count_rank'].unique(), reverse=True)[:top_N]

    # Filter the sorted and ranked results for the top 5 ranks.
    df_top_queries = df_filtered_page_views_groupby_query.loc[df_filtered_page_views_groupby_query['count_rank']
                                                            .isin(lst_top_query_count_ranks),
                                                            ['query', 'count']]

    return(df_top_queries)

In [5]:
# Create function to compute average duration for all queries.
def get_session_average_duration(df_page_views):

    # Assumption: Search session is defined as the duration of time between when a user issues a search
    # for a given keyword and when the user clicks on the last referral from the same keyword.

    # For the sake of this analysis, if a user issues another search,
    # the analysis infers that a new session has been started.

    # Add query field to page_views data frame.
    df_page_views = set_page_views_query(df_page_views)

    # Compute min and max timestamp for each session (grouped by cid and query).
    df_page_views_groupby_session = df_page_views.groupby(['cid', 'query'])['timestamp'].agg(['min', 'max'])

    # Add duration field.
    df_page_views_groupby_session['duration'] = df_page_views_groupby_session['max'] - df_page_views_groupby_session['min']

    return(round(df_page_views_groupby_session['duration'].mean(), 3))

In [6]:
print('')
print('The top ' + str(top_N) + ' most frequently issued queries (including query counts):')
print(get_top_query_counts(get_page_views(csv_path), 'issued', top_N))

print('')
print('The top ' + str(top_N) + ' most frequently clicked queries (including click counts):')
print(get_top_query_counts(get_page_views(csv_path), 'referral', top_N))

print('')
print('The average length (in seconds) of a search session (across all sessions):')
print(get_session_average_duration(get_page_views(csv_path)))


The top 5 most frequently issued queries (including query counts):
                   query  count
0                   crop     21
1             entosphere     19
2                Gaudete     17
3               operette     16
4               kerchunk     16
5            hemosalpinx     16
6            coachmaking     16
7          contrabandage     16
8             beaconless     16
9   roentgenographically     15
10         preconfinedly     15
11               archaic     15
12          trigonometry     15
13               Natalia     15
14             Argasidae     15

The top 5 most frequently clicked queries (including click counts):
           query  count
0           crop     58
1    hemosalpinx     47
2       kerchunk     46
3     entosphere     45
4  contrabandage     44
5        Natalia     44

The average length (in seconds) of a search session (across all sessions):
39.855
